# Введение в обработку естественного языка
## Урок 3. Embedding word2vec fasttext

Скачиваем датасет

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

Или данные можно скачать тут https://github.com/IlyaGusev/gazeta

Пример работы с ним:

from corus import load_ods_gazeta path = 'gazeta.csv.gz' records = load_ods_gazeta(path) next(records)

что надо сделать:

на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
Проверить насколько хорошо работают подходы

In [4]:
import string
import pickle
import annoy
import numpy as np
import pandas as pd

from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
from gensim.models import Word2Vec, FastText

In [6]:
#!wget https://www.dropbox.com/s/cmpfvzxdknkeal4/gazeta_jsonl.tar.gz

In [7]:
#!tar -xzvf gazeta_jsonl.tar.gz 

In [8]:
df_train = pd.read_json(path_or_buf='./gazeta_jsonl/gazeta_train.jsonl', lines=True)
df_val = pd.read_json(path_or_buf='./gazeta_jsonl/gazeta_val.jsonl', lines=True)
df_test = pd.read_json(path_or_buf='./gazeta_jsonl/gazeta_test.jsonl', lines=True)

df = pd.concat([df_train, df_val, df_test], ignore_index=True)
df.head()

,url,text,title,summary,date
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01


In [9]:
text_lengths = df['text'].apply(len)
summary_lengths = df['summary'].apply(len)

print(f'Средняя длина текста статьи - {np.average(text_lengths):.0f} слов.')
print(f'Средняя длина summary - {np.average(summary_lengths):.0f} слов.')

Средняя длина текста статьи - 4519 слов.
Средняя длина summary - 312 слов.


In [10]:
morpher = MorphAnalyzer()
stopwords = set(get_stop_words("ru"))
punkt = set(string.punctuation)

def preprocess_txt(line, morpher=morpher, sw=stopwords, punkt=punkt):
    spls = "".join(i for i in line.strip() if i not in punkt).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

df['preprocessed_summary'] = df['summary'].apply(preprocess_txt)
df.head()

,url,text,title,summary,date,preprocessed_summary
0,https://www.gazeta.ru/financial/2011/11/30/385...,«По итогам 2011 года чистый отток может состав...,Прогноз не успевает за оттоком,"В 2011 году из России уйдет $80 млрд, считают ...",2011-11-30 18:33:39,"[2011, россия, уйти, 80, млрд, считать, минэко..."
1,https://www.gazeta.ru/business/2013/01/24/4939...,Российское подразделение интернет-корпорации G...,Google закончил поиск,"Юлия Соловьева, экс-директор холдинга «Профмед...",2013-01-24 18:20:09,"[юлия, соловьёв, эксдиректор, холдинг, «профме..."
2,https://www.gazeta.ru/social/2018/02/06/116393...,Басманный районный суд Москвы вечером 6 феврал...,«Фигуранты дела могут давить на свидетелей»,Суд арестовал на два месяца четверых экс-чинов...,2018-02-06 21:21:14,"[суд, арестовать, месяц, четверо, эксчиновник,..."
3,https://www.gazeta.ru/business/2013/06/21/5388...,Как повлияло вступление в ВТО на конкурентносп...,«С последних традиционно «отжимают» больше»,Мнения предпринимателей по поводу вступления в...,2013-06-21 17:43:50,"[мнение, предприниматель, повод, вступление, в..."
4,https://www.gazeta.ru/culture/2014/12/27/a_636...,К третьему сезону «Голос» на Первом канале ста...,Третий «Голос» за Градского,На Первом канале завершился третий сезон шоу «...,2014-12-27 01:10:01,"[канал, завершиться, сезон, шоу, «голос», побе..."


In [13]:
window_size = 5
vector_size = 300

modelW2V = Word2Vec(sentences=df['preprocessed_summary'], 
                    vector_size=vector_size, window=window_size, 
                    min_count=1)
modelFT = FastText(sentences=df['preprocessed_summary'], 
                   vector_size=vector_size, window=window_size, 
                   min_count=1)
     

In [14]:
w2v_index = annoy.AnnoyIndex(vector_size, 'angular')
ft_index = annoy.AnnoyIndex(vector_size, 'angular')

index_map = {}
counter = 0


for i in range(len(df['preprocessed_summary'])):
    n_w2v = 0
    n_ft = 0
    index_map[i] = df['text'][i]
    
    vector_w2v = np.zeros(vector_size)
    vector_ft = np.zeros(vector_size)
    for word in df['preprocessed_summary'][i]:
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    if n_ft > 0:
        vector_ft = vector_ft / n_ft
    w2v_index.add_item(i, vector_w2v)
    ft_index.add_item(i, vector_ft)


w2v_index.build(10)
ft_index.build(10)

True

In [15]:
def get_response(question, index, model, index_map, nn=5):
    question = preprocess_txt(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, nn, )
    return [index_map[i] for i in answers]
     

In [27]:
TEXT = "личная жизнь звёзд"

In [28]:
get_response(TEXT, w2v_index, modelW2V, index_map)

["В пятницу в бар-кафе Dome прошла закрытая вечеринка, приуроченная к проходящему в эти дни кинофестивалю 2morrow. Героиней этого вечера стала Джули Круз — певица, актриса и муза Дэвида Линча , потусторонний голос которой можно услышать в «Синем бархате» и «Твин-Пикс». Круз уже в третий раз приезжает в Москву и в прошлые приезды она представляла публике ту музыку, которой занимается сегодня. Однако в этот вечер Круз представила зрителям короткую получасовую программу из знакомых киноманам номеров — «The World Spins», «Rockin' Back Inside My Heart» и т. д. После выступления с певицей побеседовал корреспондент «Парка культуры». — Вы на фестивале с первого дня. Уже посмотрели что-то? Какое впечатление он произвел на вас в целом? — Здесь очень забавно. На всех фестивалях, где мне приходилось петь, я всегда очень нервничала, переживала. Я всегда очень серьезно отношусь к выступлению, расслабиться могу, только когда оно окончено. В этом и разница, здесь гораздо спокойнее. Я как-то пела на Ка

In [29]:
get_response(TEXT, ft_index, modelFT, index_map)

["Во время пребывания на фестивале складывалось ощущение, что наградами здесь интересуются постольку поскольку. В обстановке всеобщего братания в очередях на показы (про систему распределения билетов можно сочинить отдельный триллер или драму) и на многочисленных вечеринках сложно сфокусироваться на отдельном конкурсе, а удержать в голове все разом так и вовсе невозможно. Большинство хаотически перемещается по каким-то собственным кино-маршрутам; самый распространенный вопрос: «а вы здесь с каким фильмом?». В этом весь «Сандэнс», во время которого многочисленные молодые кинематографисты со всего света в возрасте от 20 до 60 встречаются, обсуждают свои проекты, обмениваются контактами и ощущением того, что все возможно: и снять фильм без денег, и найти деньги на него, и начать актерскую карьеру на пенсии. Но при всем разнообразии программ мы выбрали в качестве главной американский конкурс художественных фильмов, посмотрев все 16 представленных в нем претендентов на призы, что из всех вс

In [19]:
TEXT = "географы открыли новый континент"

In [20]:
get_response(TEXT, w2v_index, modelW2V, index_map)

['При запуске на рынок новых моделей многие автопроизводители растягивают удовольствие, объявляя дебют автомобилей с разным типом кузова на разных международных выставках и начиная продажи в разное время. Так, в Париже автомобилями в кузове универсал пополнили модельную линейку Opel и Volvo. Французы же на домашнем мотор-шоу мелочиться не стали: новый флагман Peugeot 508 представлен в виде седана и универсала. Второе поколение Citroen C4, в соответствии с нынешней концепцией марки, разделилось на две модели, показанные одновременно. Азиатские автопроизводители показали компактные модели: Toyota выводит на европейский рынок микровэн Verso S, Honda дополнила Jazz гибридной модификацией, а корейский Hyundai представил компактвэн ix20, пришедший на смену модели Matrix, производившейся с 2001 года. Помимо демонстрации предсерийного «горячего хетчбэка» Ford Focus ST, начало производства которого запланировано на 2012 год, американский концерн обнародовал всю номенклатуру двигателей и комплек

In [21]:
get_response(TEXT, ft_index, modelFT, index_map)

['В четверг, 12 ноября, с блока детских мультфильмов, познавательной передачи «100 великих…» и первого фильма бондианы — «Доктора Ноу» с Шоном Коннери — начал работу телеканал «Че». Он вещает на той частоте, которую до сих пор занимал канал «Перец». Новый канал обращен совсем к другой аудитории. Ключевой идеей канала стало отражение образа жизни «настоящего Человека», а сам «Че» станет каналом про мужчин. «Это канал про мужчин, которые интересуются историей страны, уважают свою профессию, любят семью, дорожат дружбой и ценят качественные фильмы и сериалы», — сказано в сообщении канала. Название «Че» прежде всего отсылает к слову «человек», а также к таким понятиям, как «увлеЧЕния», «развлеЧЕния» и «ЧЕмпионы». Концепция подкреплена цитатами из классики — «В Че все должно быть прекрасно» или «В любой ситуации оставайся Че». Не обошлось, разумеется, и без «Че Гевары». «Сегодня мы только начали развитие и формирование образа канала «Че». 12 ноября – это отправная точка, после которой мы бу